# Rigorous Thermodynamic Decomposition of Salt Effects on the Polymerization of Polyethylene Glycol
Stefan Hervø-Hansen<sup>a,*</sup>, Jan Heyda<sup>b,*</sup>, and Nobuyuki Matubayasi<sup>a,*</sup>.<br><br>
<sup>a</sup> Division of Chemical Engineering, Graduate School of Engineering Science, Osaka University, Toyonaka, Osaka 560-8531, Japan.<br>
<sup>b</sup> Department of Physical Chemistry, University of Chemistry and Technology, Prague CZ-16628, Czech Republic.<br>
<sup>*</sup> To whom correspondence may be addressed: stefan@cheng.es.osaka-u.ac.jp, heydaj@vscht.cz, and nobuyuki@cheng.es.osaka-u.ac.jp.

## Part 1: Simulations


### Introduction


### Methods & Materials
Molecular dynamics simulations are conducted using the openMM (7.7.0)[<sup>1</sup>](#fn1) software package modded with the parmed[<sup>2</sup>](#fn2) package and be found in the [Part 1 Jupyter notebook](Simulations.ipynb). For the simulation of PEG a CHARMM derived force field (C35r) was utilized, which has previously been able to reproduce hydrodynamic radii and shape anisotropy of PEG.[<sup>3</sup>](#fn3) The PEG force field was employed in combination with the SPC/E force field for water[<sup>4</sup>](#fn4) and optimized ion parameters for sodium thiocyanate and sodium chloride.[<sup>5,</sup>](#fn5)[<sup>6</sup>](#fn6)
The isothermal-isobaric ensemble will be sampled using a combination of a "Middle" discretization Langevin leap-frog integrator[<sup>7</sup>](#fn7) and a Monte Carlo barostat[<sup>8,</sup>](#fn8)[<sup>9</sup>](#fn9). The trajectories was analyzed using MDtraj[<sup>10</sup>](#fn10) for structural properties, while ERmod[<sup>11</sup>](#fn11) be utilized for the calculation of solvation free energies and can be found in the [Part 2 Jupyter notebook](Analysis.ipynb).

### References
1. <span id="fn1"> P. Eastman, et al., OpenMM 7: Rapid development of high performance algorithms for molecular dynamics. PLoS Comput Biol 13, e1005659 (2017).</span><br>
2. <span id="fn2"> https://github.com/ParmEd/ParmEd </span><br>
3. <span id="fn3"> H. Lee, R. M. Venable, A. D. MacKerell Jr., R. W. Pastor, Molecular Dynamics Studies of Polyethylene Oxide and Polyethylene Glycol: Hydrodynamic Radius and Shape Anisotropy. Biophysical Journal 95, 1590–1599 (2008). </span><br>
4. <span id="fn4"> H. J. C. Berendsen, J. R. Grigera, T. P. Straatsma, The missing term in effective pair potentials. J. Phys. Chem. 91, 6269–6271 (1987). </span><br>
5. <span id="fn5"> T. Křížek, et al., Electrophoretic mobilities of neutral analytes and electroosmotic flow markers in aqueous solutions of Hofmeister salts. ELECTROPHORESIS 35, 617–624 (2014). </span><br>
6. <span id="fn6"> J. Heyda, J. C. Vincent, D. J. Tobias, J. Dzubiella, P. Jungwirth, Ion Specificity at the Peptide Bond: Molecular Dynamics Simulations of N-Methylacetamide in Aqueous Salt Solutions. J. Phys. Chem. B 114, 1213–1220 (2009). </span><br>
7. <span id="fn7"> Z. Zhang, X. Liu, K. Yan, M. E. Tuckerman, J. Liu, Unified Efficient Thermostat Scheme for the Canonical Ensemble with Holonomic or Isokinetic Constraints via Molecular Dynamics. J. Phys. Chem. A 123, 6056–6079 (2019). </span><br>
8. <span id="fn8"> K.-H. Chow, D. M. Ferguson, Isothermal-isobaric molecular dynamics simulations with Monte Carlo volume sampling. Computer Physics Communications 91, 283–289 (1995). </span><br>
9. <span id="fn9"> J. Åqvist, P. Wennerström, M. Nervall, S. Bjelic, B. O. Brandsdal, Molecular dynamics simulations of water and biomolecules with a Monte Carlo constant pressure algorithm. Chemical Physics Letters 384, 288–294 (2004). </span><br>
10. <span id="fn10"> R. T. McGibbon, et al., MDTraj: A Modern Open Library for the Analysis of Molecular Dynamics Trajectories. Biophysical Journal 109, 1528–1532 (2015) </span><br>
11. <span id="fn11"> S. Sakuraba, N. Matubayasi, Ermod: Fast and versatile computation software for solvation free energy with approximate theory of solutions. J. Comput. Chem. 35, 1592–1608 (2014). </span><br>


## Import of Python Modules & Auxiliary Functions

In [ ]:
# Notebook dependent libs
import numpy as np
import matplotlib.pyplot as plt
import os, re, time
from shutil import copyfile
from distutils.spawn import find_executable
import parmed as pmd
from openmm import app
import openmm as mm
import mdtraj as md

# Simulation specific libs
# from openmm import app
# import openmm as mm
# import mdtraj as md

# Check for external programs
if None in [find_executable('packmol')]:
    print('WARNING: External program missing!')
    
# Physical constants & useful conversions
Na = 6.02214076e23 # Avogadro constant [mol]
liter_to_cubeangstrom = 1e27


homedir = !pwd
homedir = homedir[0]
print(homedir)

## Molecular Dynamics Simulations
To provide a rigorous thermodynamic description of the salt effects on PEG of various lengths, as described in the paper, we adopt an unconventional simulation strategy to calculate the difference in chemical potential of PEG upon the addition of salt. The simulation strategy is visualized below.

<img style="float: center;" src="Figures/Simulation_strategy.jpeg" title="Simulation flow" />

In specific, the simulation strategy contains three steps:
1. Generation of a PEG conformational ensemble in the reference state being pure water.
2. Generation of solvent configurations in the absences ($\lambda=0$) of PEG.
3. Generation of solvent configurations in the presences ($\lambda=1$) of a fixed PEG configuration.

The steps of the simulation strategy are outlined throughout the Jupyter Notebook.

### Simulation settings
The number of steps and the output frequency of the simulations are controlled via the `states` variable. The dictionary contains 3 entries, namely `conf`, `sol`, and `ref` which are referring to the simulations for generating the PEG conformational ensemble in water, the simulation of the solvated state ($\lambda=1$) with frozen PEG, and the reference state ($\lambda=0$) in which PEG is absent, respectively. 

The number of conformations chosen via `NConfs` is picked via linear spaced sampling from the conformational ensemble generated.


### OpenMM settings & setup
The notebook is designed to be fully automated in the construction of simulation input files for OpenMM using the python API. If you wish to edit the default OpenMM script one simply needs to edit the variable `openmm_script`. For example, one can edit the integration scheme and its parameters set in the variable `integrator` as well as edit the barostat as currently determined from `mm.MonteCarloBarostat`. Additionally, one may change the non-bonded methods and their cutoffs in the `system` variable with the option of adding Lennard-Jones switching functions via the `forces` variable. Finally, one may edit the number of minimization (`sim.minimizeEnergy`) and equilibration steps conducted as well as choose whether the simulation should be conducted on GPUs or CPUs via the `platform` and `properties` variables.

### Submit script
Due to the larger amount of simulations, the notebook can write submission script for servers employing job scheduling. The notebook was origionally run on a linux machine utilizing PBS (for a quick guide see [here](https://latisresearch.umn.edu/creating-a-PBS-script)). However, the bash script containing the submission code may be edited to utilize Slurm instead (documentation [here](https://slurm.schedmd.com)) by changing the variable `submit_script` and by executing the commands `!sbatch submit.pbs`, `!sbatch submit_sol.pbs`, and `!sbatch submit_ref.pbs` instead of `qsub`.

In [ ]:
states = { # State of simulations, (outFreq is steps per frame)
          'conf':{'Nsteps': 100000000, 'OutFreq': 5000}, # 200 nanoseconds, 20000 frames
          'sol': {'Nsteps':  50000000, 'OutFreq': 1000}, # 100 nanoseconds, 50000 frames
          'ref': {'Nsteps':  25000000, 'OutFreq': 1000}, #  50 nanoseconds, 25000 frames
         }

nmers = [2, 4, 6, 8, 15, 36] # PEG polymer length
nmers = [8]                  # TEMPORARY OVERWRITE
NConfs = 25                  # Number of fixed PEG conformation used in chem. pot. calculation

Nparticles = {       # Number of PEG and water molecules. Salt is calculated based on concentration input
    'PEG': 1,
    'Water': 10000,
}

# Approximate concentrations of salt (in Molar) under which the structual sampling is conducted.
salt_reference_concentrations = { # P1 and P2 are the perturbations that will be added to the salt concentration
    0.00: {'P0': 0.00, 'P1':  0.10, 'P2': 0.20},
#    0.20: {'P0': 0.20, 'P1': -0.10, 'P2': 0.10},
#    1.00: {'P0': 1.00, 'P1': -0.10, 'P2': 0.10},
}

GENERATE_SOLUTE     = True   # Generate the structual ensemble of PEG given a specific reference salt concentration
GENERATE_REFERENCES = False   # Generate the reference conditions
GENERATE_SOLUTIONS  = False  # Generate the solution conditions

salts = { # Types of salt added to the simulations.
         'NaCl'   : {'Cation': 'Na' , 'Anion': 'Cl' },
#         'NaSCN'  : {'Cation': 'Na' , 'Anion': 'SCN'},
}

### Step 1. Generation of Solute Configurations

#### Construction structure files

In [ ]:
%cd -q $homedir

packmol_script="""
tolerance 2.0
filetype pdb
output PEG_{nmer}_{cation}{anion}_{conc}.pdb

add_box_sides 1.0

structure {homedir}/PDB_files/PEO-{nmer}-mer.pdb
        number {N_PEG}
        fixed 35. 35. 35. 0. 0. 0.
        centerofmass
end structure

{salt}structure {homedir}/PDB_files/{anion}.pdb
{salt}        number {N_anion}
{salt}        inside cube 0. 0. 0. 70.
{salt}end structure

{salt}structure {homedir}/PDB_files/{cation}.pdb
{salt}        number {N_cation}
{salt}        inside cube 0. 0. 0. 70.
{salt}end structure
"""

for nmer in nmers:
    nmerdir = 'PEG{}mer'.format(nmer)
    for saltdir, salt in salts.items():
        for saltreference in salt_reference_concentrations:
            saltreference = '{0:.2f}'.format(saltreference)
            %cd -q $homedir/Simulations/$nmerdir/$saltdir/$saltreference/Solute

            # Packmol Input
            with open('packmol.in', 'w') as text_file:
                # Fix for no salt
                if float(saltreference) == 0:
                    saltFix='#'
                    Nions = 0
                else:
                    saltFix=''
                    Nions  = round(float(saltreference) * 70**3 * Na / liter_to_cubeangstrom)
                text_file.write(packmol_script.format(N_PEG=Nparticles['PEG'], nmer=nmer, salt=saltFix,
                                                      homedir=homedir, cation=salt['Cation'], anion=salt['Anion'],
                                                      N_anion=Nions, N_cation=Nions, conc=saltreference))
            text_file.close()
            !packmol < packmol.in
            
            print('Wrote initial configurations and topology files to'+os.getcwd())

#### Molecular dynamics setup using OpenMM

In [ ]:
%cd -q $homedir

openmm_script="""# Imports
import sys
import os
import openmm as mm
from openmm import app
from openmm import unit as u
from mdtraj.reporters import XTCReporter

print('Loading initial configuration and toplogy')
pdb = app.PDBFile('PEG_{nmer}_{salt}_{conc}.pdb')
forcefield = app.ForceField('{homedir}/Force_fields/peg.xml',
                            '{homedir}/Force_fields/spce.xml',
                            '{homedir}/Force_fields/SCN.xml',
                            '{homedir}/Force_fields/ions.xml')
                         
modeller = app.Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens(forcefield, pH=7.0)
modeller.addSolvent(forcefield, model='spce', numAdded=10000, neutralize=False)

app.PDBFile.writeFile(modeller.topology, modeller.positions, open('PEG_{nmer}_{salt}_{conc}_hydrated.pdb', 'w'))

# Creating system
print('Creating OpenMM System')
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, ewaldErrorTolerance=0.0005, switchDistance=1*u.nanometer,
                                 nonbondedCutoff=1.2*u.nanometers, constraints=app.HBonds, rigidWater=True)

# Calculating total mass of system
total_mass = u.sum([system.getParticleMass(i) for i in range(system.getNumParticles())])
        
# Temperature-coupling by leap frog (BAOAB) Langevin integrator (NVT)
integrator = mm.LangevinMiddleIntegrator(298.15*u.kelvin, 1.0/u.picoseconds, 2.0*u.femtoseconds)

# Pressure-coupling by a Monte Carlo Barostat (NPT)
system.addForce(mm.MonteCarloBarostat(1*u.bar, 298.15*u.kelvin, 25))

platform = mm.Platform.getPlatformByName('CUDA')
properties = {{'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0'}}

# Create the Simulation object
sim = app.Simulation(modeller.topology, system, integrator, platform, properties)

# Set the particle positions
sim.context.setPositions(modeller.positions)

# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(tolerance=1*u.kilojoule/u.mole, maxIterations=1000000)
    
# Draw initial MB velocities
sim.context.setVelocitiesToTemperature(298.15*u.kelvin)

# Equlibrate simulation
print('Equilibrating...')
sim.step(500000)  # 500000*2 fs = 1.0 ns

# Set up the reporters
sim.reporters.append(app.StateDataReporter('output.dat', {outFreq}, totalSteps={Nsteps}+500000,
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True, volume=True, density=True,
    systemMass=total_mass, remainingTime=True, speed=True, separator='\t'))

# Set up trajectory reporter
sim.reporters.append(XTCReporter('trajectory.xtc', reportInterval={outFreq}, append=False))

# Run dynamics
print('Running dynamics! (NPT)')
sim.step({Nsteps})
"""           

N_simulations = 0
if GENERATE_SOLUTE:
    for nmer in nmers:
        nmerdir = 'PEG{}mer'.format(nmer)
        for saltdir, salt in salts.items():
            for saltreference in salt_reference_concentrations:
                saltreference = '{0:.2f}'.format(saltreference)
                %cd -q $homedir/Simulations/$nmerdir/$saltdir/$saltreference/Solute
                
                with open('openMM.py', 'w') as text_file:
                    text_file.write(openmm_script.format(homedir=homedir, Nsteps=states['conf']['Nsteps'],
                                                         nmer=nmer, outFreq=states['conf']['OutFreq'],
                                                         salt=saltdir, conc=saltreference))
                text_file.close()
                print('Wrote run_openMM.py files to '+os.getcwd())
                N_simulations+=1

print('Simulations about to be submitted: {}'.format(N_simulations))

#### Submit script

In [ ]:
%cd -q $homedir

submit_script="""#!/bin/bash
#PJM -L "rscunit=ito-b"
#PJM -L "rscgrp=ito-g-1"
#PJM -L "vnode=1"
#PJM -L "vnode-core=9"
#PJM -L "elapse=0168:00:00"
#PJM -e run.err
#PJM -o run.out

source ~/.bashrc
source ~/.bash_profile

cd {path}

python openMM.py"""

if GENERATE_SOLUTE:
    for nmer in nmers:
        nmerdir = 'PEG{}mer'.format(nmer)
        for saltdir, salt in salts.items():
            for saltreference in salt_reference_concentrations:
                saltreference = '{0:.2f}'.format(saltreference)
                %cd -q $homedir/Simulations/$nmerdir/$saltdir/$saltreference/Solute
                
                with open('submit.pbs', 'w') as text_file:
                    text_file.write(submit_script.format(nmer=nmer, path=os.getcwd()))
                text_file.close()
                !pjsub submit.pbs
                time.sleep(1) # Safety in submission of jobs: can cause problems if too fast

#### Post simulation trajectory processing

In [ ]:
%cd -q $homedir

if GENERATE_SOLUTE:
    for nmer in nmers:
        nmerdir = 'PEG{}mer'.format(nmer)
        for saltdir, salt in salts.items():
            for saltreference in salt_reference_concentrations:
                saltreference = '{0:.2f}'.format(saltreference)
                %cd -q $homedir/Simulations/$nmerdir/$saltdir/$saltreference/Solute
                
                pdb = md.load_pdb('PEG_{nmer}_{salt}_{conc}_hydrated.pdb'.format(nmer=nmer, salt=saltdir, conc=saltreference))
                PEG_indices = pdb.topology.select('not water')
                traj = md.load_xtc('trajectory.xtc',
                                   top='PEG_{nmer}_{salt}_{conc}_hydrated.pdb'.format(nmer=nmer, salt=saltdir, conc=saltreference), atom_indices=PEG_indices)
                traj.save_xtc('trajectory_dry.xtc'.format(nmer), force_overwrite=True)

### Step 2. Generation of Solvent Configurations - Reference System (λ = 0)
#### Generation of initial configuration using Packmol

In [ ]:
%cd -q $homedir

packmol_script="""
tolerance 2.0
filetype pdb
output {cation}{anion}_{conc}.pdb

add_box_sides 1.0

structure {homedir}/PDB_files/{anion}.pdb
        number {N_anion}
        inside cube 0. 0. 0. 70.
end structure

structure {homedir}/PDB_files/{cation}.pdb
        number {N_cation}
        inside cube 0. 0. 0. 70.
end structure
"""

concentrations = []

if GENERATE_REFERENCES:
    for refs, settings in salt_reference_concentrations.items():
        concentrations.append(settings['P0'])
        concentrations.append(round(settings['P0']+settings['P1'], ndigits=2))
        concentrations.append(round(settings['P0']+settings['P2'], ndigits=2))
    
    concentrations = list(set(concentrations))
    
    for i, (saltdir, salt) in enumerate(salts.items()):
        for conc in concentrations:
            conc = '{0:.2f}'.format(conc)
            if conc == '0.00' and i > 0:   # Do only neat water ones
                continue

            elif conc == '0.00' and i == 0: # For first encounter of neat water
                %cd -q $homedir/Simulations/References/No_salt
                modeller = app.Modeller(app.Topology(), [])
            
            else: # For any system containing salt
                %cd -q $homedir/Simulations/References/$saltdir/$conc
                Nions  = round(float(conc) * 70**3 * Na / liter_to_cubeangstrom)
                with open('packmol.in', 'w') as text_file:
                    text_file.write(packmol_script.format(homedir=homedir, N_anion=Nions, N_cation=Nions, conc=conc,
                                                          cation=salt['Cation'], anion=salt['Anion']))
                text_file.close()
                !packmol < packmol.in
                pdb = app.PDBFile('{salt}_{conc}.pdb'.format(salt=saltdir, conc=conc))
                modeller = app.Modeller(pdb.topology, pdb.positions)
                
            forcefield = app.ForceField(homedir+'/Force_fields/peg.xml', homedir+'/Force_fields/spce.xml',
                                        homedir+'/Force_fields/SCN.xml', homedir+'/Force_fields/ions.xml')
            modeller.addSolvent(forcefield, model='spce', numAdded=Nparticles['Water'], neutralize=False)
            app.PDBFile.writeFile(modeller.topology, modeller.positions, file=open('{salt}_{conc}.pdb'.format(salt=saltdir, conc=conc), 'w'))
            
            print('Wrote initial configurations to'+os.getcwd())

#### Molecular dynamics setup using OpenMM

In [ ]:
%cd -q $homedir

openmm_script="""# Imports
import sys
import os
import openmm as mm
from openmm import app
from openmm import unit as u
from mdtraj.reporters import XTCReporter

print('Loading initial configuration and toplogy')
pdb = app.PDBFile('{salt}_{conc}.pdb')
forcefield = app.ForceField('{homedir}/Force_fields/peg.xml',
                            '{homedir}/Force_fields/spce.xml',
                            '{homedir}/Force_fields/SCN.xml',
                            '{homedir}/Force_fields/ions.xml')


# Creating system
print('Creating OpenMM System')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME, ewaldErrorTolerance=0.0005, switchDistance=1*u.nanometer,
                                 nonbondedCutoff=1.2*u.nanometers, constraints=app.HBonds, rigidWater=True)

# Calculating total mass of system
total_mass = u.sum([system.getParticleMass(i) for i in range(system.getNumParticles())])
        
# Temperature-coupling by leap frog (BAOAB) Langevin integrator (NVT)
integrator = mm.LangevinMiddleIntegrator(298.15*u.kelvin, 1.0/u.picoseconds, 2.0*u.femtoseconds)

# Pressure-coupling by a Monte Carlo Barostat (NPT)
system.addForce(mm.MonteCarloBarostat(1*u.bar, 298.15*u.kelvin, 25))

platform = mm.Platform.getPlatformByName('CUDA')
properties = {{'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0'}}

# Create the Simulation object
sim = app.Simulation(pdb.topology, system, integrator, platform, properties)

# Set the particle positions
sim.context.setPositions(pdb.positions)

# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(tolerance=1*u.kilojoule/u.mole, maxIterations=1000000)
    
# Draw initial MB velocities
sim.context.setVelocitiesToTemperature(298.15*u.kelvin)

# Equlibrate simulation
print('Equilibrating...')
sim.step(500000)  # 500000*2 fs = 1.0 ns

# Set up the reporters
sim.reporters.append(app.StateDataReporter('output.dat', {outFreq}, totalSteps={Nsteps}+500000,
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True, volume=True, density=True,
    systemMass=total_mass, remainingTime=True, speed=True, separator='\t'))

# Set up trajectory reporter
sim.reporters.append(XTCReporter('trajectory.xtc', reportInterval={outFreq}, append=False))

# Run dynamics
print('Running dynamics! (NPT)')
sim.step({Nsteps})
"""

N_simulations = 0
if GENERATE_REFERENCES:
    for i, saltdir in enumerate(salts):
        for conc in concentrations:
            conc = '{0:.2f}'.format(conc)
            if conc == '0.00' and i > 0:
                continue
            elif conc == '0.00' and i == 0:
                %cd -q $homedir/Simulations/References/No_salt
            else:
                %cd -q $homedir/Simulations/References/$saltdir/$conc
                
            with open('openMM.py', 'w') as text_file:
                text_file.write(openmm_script.format(Nsteps=states['ref']['Nsteps'], homedir=homedir, conc=conc,
                                                    outFreq=states['ref']['OutFreq'], salt=saltdir))
            text_file.close()
            N_simulations+=1
            print('Wrote run_openMM.py files to '+os.getcwd())

print('Simulations about to be submitted: {}'.format(N_simulations))

#### Submit script

In [ ]:
%cd -q $homedir

submit_script="""#!/bin/bash
#PJM -L "rscunit=ito-b"
#PJM -L "rscgrp=ito-g-1"
#PJM -L "vnode=1"
#PJM -L "vnode-core=9"
#PJM -L "elapse=0168:00:00"
#PJM -e run.err
#PJM -o run.out

source ~/.bashrc
source ~/.bash_profile

cd {path}

python openMM.py"""

if GENERATE_REFERENCES:
    for i, saltdir in enumerate(salts):
        for conc in concentrations:
            conc = '{0:.2f}'.format(conc)
            if conc == '0.00' and i > 0:
                continue
            elif conc == '0.00' and i == 0:
                %cd -q $homedir/Simulations/References/No_salt
            else:
                %cd -q $homedir/Simulations/References/$saltdir/$conc
            
            with open('submit.pbs', 'w') as text_file:
                text_file.write(submit_script.format(path=os.getcwd()))
            !pjsub submit.pbs
            time.sleep(1) # Safety in submission of jobs: can cause problems if too fast

### Step 3. Generation of Solvent Configurations at Constant Solute Configuration - Solution System (λ = 1)
#### Construction of topology and structure files
Fully automated construction of topologies files in Amber format and initial configurations using packmol. No major important parameters to edit in the following code.

In [ ]:
%cd -q $homedir

packmol_script="""
tolerance 2.0
filetype pdb
output PEG_{nmer}_{cation}{anion}.pdb

add_box_sides 0.1

structure conf.pdb
        number 1
        fixed 35. 35. 35. 0. 0. 0.
        centerofmass
end structure

structure {homedir}/PDB_files/{anion}.pdb
        number {N_anion}
        inside cube 0. 0. 0. 70.
end structure

structure {homedir}/PDB_files/{cation}.pdb
        number {N_cation}
        inside cube 0. 0. 0. 70.
end structure
"""

traj_indices = np.rint(np.linspace(0, (states['conf']['Nsteps']//states['conf']['OutFreq'])-1, NConfs)).astype(np.int64)


for nmer in nmers:
    nmerdir = 'PEG{}mer'.format(nmer)
    for saltdir, salt in salts.items():
        for conc, perturb in salt_reference_concentrations.items():
            conc = '{0:.2f}'.format(conc)
            %cd -q $homedir/Simulations/$nmerdir/$saltdir/$conc
            
            peg_traj = md.load_xtc('Solute/trajectory.xtc',
                                   top='Solute/PEG_{nmer}_{salt}_{conc}_hydrated.pdb'.format(nmer=nmer, salt=saltdir, conc=conc),
                                   stride=states['conf']['Nsteps']//(states['conf']['OutFreq']*NConfs))
            
            assert len(peg_traj) == NConfs, 'The number of frames loaded is inconsistant with the number of desired conformations'

            for P in ['P0','P1', 'P2']:
                %cd -q $homedir/Simulations/$nmerdir/$saltdir/$conc
                for conf in range(NConfs):
                    if P == 'P0': # Tested and works
                        peg_traj[conf].save('P0/{conf}/PEG_{nmer}_{salt}.pdb'.format(nmer=nmer, salt=saltdir, conf=conf))
                        pdb = app.PDBFile('P0/{conf}/PEG_{nmer}_{salt}.pdb'.format(nmer=nmer, salt=saltdir, conf=conf))
                        modeller = app.Modeller(pdb.topology, pdb.positions)
                        forcefield = app.ForceField(homedir+'/Force_fields/peg.xml', homedir+'/Force_fields/spce.xml',
                                                    homedir+'/Force_fields/SCN.xml', homedir+'/Force_fields/ions.xml')                        
                    
                    elif perturb[P] < 0 and P != 'P0':
                        mol = peg_traj[conf]
                        
                        Nions  = abs(round(float(perturb[P]) * 70**3 * Na / liter_to_cubeangstrom))
                        
                        ions_to_remove = []
                        for residue in mol.topology.residues:
                            if residue.name in [salt['Anion']]:
                                pairs = mol.topology.select_pairs('(resname PGH) or (resname PGM) or (resname PGT)',
                                                                'resid {}'.format(residue.index))
                                distances = md.compute_distances(t, pairs)[0]
                                if not np.any(distances<0.5):
                                    ions_to_remove.append([atom.index for atom in residue.atoms])
                            if len(ions_to_remove) == Nions:
                                break
                        for residue in mol.topology.residues:
                            if residue.name in [salt['Cation']]:
                                pairs = mol.topology.select_pairs('(resname PGH) or (resname PGM) or (resname PGT)',
                                                                'resid {}'.format(residue.index))
                                distances = md.compute_distances(t, pairs)[0]
                                if not np.any(distances<0.5):
                                    ions_to_remove.append([atom.index for atom in residue.atoms])
                            if len(ions_to_remove) == Nions:
                                break
                        ions_to_remove = np.array(ions_to_remove).flatten()
                        mol.atom_slice(ions_to_remove, inplace=True)
                        mol.save('{P}/{conf}/PEG_{nmer}_{salt}.pdb'.format(P=P, nmer=nmer, salt=saltdir, conf=conf))
                        pdb = app.PDBFile('{P}/{conf}/PEG_{nmer}_{salt}.pdb'.format(P=P, nmer=nmer, salt=saltdir, conf=conf))
                        modeller = app.Modeller(pdb.topology, pdb.positions)
                        forcefield = app.ForceField(homedir+'/Force_fields/peg.xml', homedir+'/Force_fields/spce.xml',
                                                    homedir+'/Force_fields/SCN.xml', homedir+'/Force_fields/ions.xml')            
                        
                    elif perturb[P] > 0 and P != 'P0': # Tested and works
                        %cd -q $homedir/Simulations/$nmerdir/$saltdir/$conc/$P/$conf
                        mol = peg_traj[conf]
                        mol.remove_solvent(inplace=True)
                        mol.save('conf.pdb')
                        Nions  = round(float(perturb[P]) * 70**3 * Na / liter_to_cubeangstrom)
                        with open('packmol.in', 'w') as text_file:
                            text_file.write(packmol_script.format(homedir=homedir, N_anion=Nions, N_cation=Nions, nmer=nmer,
                                                          cation=salt['Cation'], anion=salt['Anion']))
                        text_file.close()
                        !packmol < packmol.in
                        pdb = app.PDBFile('PEG_{nmer}_{salt}.pdb'.format(nmer=nmer, salt=saltdir))
                        modeller = app.Modeller(pdb.topology, pdb.positions)
                        forcefield = app.ForceField(homedir+'/Force_fields/peg.xml', homedir+'/Force_fields/spce.xml',
                                                    homedir+'/Force_fields/SCN.xml', homedir+'/Force_fields/ions.xml')
                        modeller.addSolvent(forcefield, model='spce', numAdded=Nparticles['Water'], neutralize=False)
                        app.PDBFile.writeFile(modeller.topology, modeller.positions, file=open('PEG_{nmer}_{salt}.pdb'.format(nmer=nmer, salt=saltdir), 'w'))
                    
                # Collect it all for λ=1
                %cd -q $homedir/Simulations/$nmerdir/$saltdir/$conc
                system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, ewaldErrorTolerance=0.0005,
                                                     nonbondedCutoff=1.2*pmd.unit.nanometers, rigidWater=False)
                mol = pmd.openmm.load_topology(modeller.topology, system, modeller.positions)
                mol.save('{P}/PEG_{nmer}_{salt}.parm7'.format(P=P, nmer=nmer, salt=saltdir), overwrite=True)
                print('Wrote initial configurations and topology files to'+os.getcwd())

#### Construction of molecular dynamics input for OpenMM

In [ ]:
%cd -q $homedir
openmm_script="""# Imports
import sys
import os
import openmm as mm
from openmm import app
from openmm import unit as u
from mdtraj.reporters import XTCReporter

print('Loading initial configuration and toplogy')
pdb = app.PDBFile('PEG_{nmer}_{salt}.pdb')
forcefield = app.ForceField('{homedir}/Force_fields/peg.xml',
                            '{homedir}/Force_fields/spce.xml',
                            '{homedir}/Force_fields/SCN.xml',
                            '{homedir}/Force_fields/ions.xml')
    
# Find all PEG atoms   
PEG_atoms = []
for residue in pdb.topology.residues():
    if residue.name in ['PGH', 'PGM', 'PGT']:
        for atom in residue.atoms():
            PEG_atoms.append(atom)

# Creating system
print('Creating OpenMM System')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME, ewaldErrorTolerance=0.0005, switchDistance=1*u.nanometer,
                                 nonbondedCutoff=1.2*u.nanometers, constraints=app.HBonds, rigidWater=True)

# Calculating total mass of system
total_mass = u.sum([system.getParticleMass(i) for i in range(system.getNumParticles())])

# Freeze PEG atoms by setting mass to 0
for atom in PEG_atoms:
    system.setParticleMass(atom.index, 0.000*u.dalton)
        
# Temperature-coupling by leap frog "middle"-discretization Langevin integrator (NVT)
integrator = mm.LangevinMiddleIntegrator(0.15*u.kelvin, 100.0/u.picoseconds, 0.05*u.femtoseconds)

# Pressure-coupling by a Monte Carlo Barostat (NPT)
system.addForce(mm.MonteCarloBarostat(1*u.bar, 0.15*u.kelvin, 25))

platform = mm.Platform.getPlatformByName('CUDA')
properties = {{'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0'}}

# Create the Simulation object
sim = app.Simulation(pdb.topology, system, integrator, platform, properties)

# Set the particle positions
sim.context.setPositions(pdb.positions)

# Minimize the energy
print('Minimizing energy using Langevin dynamics at low temperature and high drag...')
sim.step(500000)  # 500000*0.05 fs = 25 ps

# Save minimized coordinates
positions = sim.context.getState(getPositions=True).getPositions()

# Change Langevin integrator and Monte Carlo barostat back to correct parameters
sim.integrator.setFriction(1.0/u.picoseconds)
sim.integrator.setStepSize(2.0*u.femtoseconds)
sim.integrator.setTemperature(298.15*u.kelvin)
for param in sim.context.getParameters():
    if 'MonteCarloTemperature' in param:
        sim.context.setParameter(param, 298.15*u.kelvin)
sim.context.setTime(0)
sim.context.reinitialize()
sim.context.setPositions(positions)

# Draw new MB velocities
sim.context.setVelocitiesToTemperature(298.15*u.kelvin)

# Equlibrate simulation
print('Equilibrating...')
sim.step(500000)  # 500000*2 fs = 1.0 ns

# Set up the reporters
sim.reporters.append(app.StateDataReporter('output.dat', {outFreq}, totalSteps={Nsteps}+500000,
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True, volume=True, density=True,
    systemMass=total_mass, remainingTime=True, speed=True, separator='\t'))

# Set up trajectory reporter
sim.reporters.append(XTCReporter('trajectory.xtc', reportInterval={outFreq}, append=False))

# Run dynamics
print('Running dynamics! (NPT)')
sim.step({Nsteps})

# Print PME information
print('''
PARTICLE MESH EWALD PARAMETERS
Separation parameter: {{}}
Number of grid points along the X axis: {{}}
Number of grid points along the Y axis: {{}}
Number of grid points along the Z axis: {{}}
'''.format(*sim.system.getForces()[3].getPMEParametersInContext(sim.context)))
"""

N_simulations = 0
for nmer in nmers:
    nmerdir = 'PEG{}mer'.format(nmer)
    for saltdir, salt in salts.items():
        for conc in salt_reference_concentrations:
            conc = '{0:.2f}'.format(conc)
            for P in ['P0','P1', 'P2']:
                for conf in range(NConfs):
                    %cd -q $homedir/Simulations/$nmerdir/$saltdir/$conc/$P/$conf
                    
                    with open('openMM.py', 'w') as text_file:
                        text_file.write(openmm_script.format(Nsteps=states['sol']['Nsteps'], homedir=homedir,
                                                             outFreq=states['sol']['OutFreq'], salt=saltdir, nmer=nmer))
                    text_file.close()
                    N_simulations+=1
                    print('Wrote run_openMM.py files to '+os.getcwd())

print('Simulations about to be submitted: {}'.format(N_simulations))

#### Submit script

In [ ]:
submit_script="""#!/bin/bash
#PJM -L "rscunit=ito-b"
#PJM -L "rscgrp=ito-g-1"
#PJM -L "vnode=1"
#PJM -L "vnode-core=9"
#PJM -L "elapse=0168:00:00"
#PJM -e run.err
#PJM -o run.out

source ~/.bashrc
source ~/.bash_profile

cd {path}

python openMM.py"""

for nmer in nmers:
    nmerdir = 'PEG{}mer'.format(nmer)
    for saltdir, salt in salts.items():
        for conc in salt_reference_concentrations:
            conc = '{0:.2f}'.format(conc)
            for P in ['P0','P1', 'P2']:
                for conf in range(NConfs):
                    %cd -q $homedir/Simulations/$nmerdir/$saltdir/$conc/$P/$conf
                
                    with open('submit.pbs', 'w') as text_file:
                        text_file.write(submit_script.format(path=os.getcwd()))
                    text_file.close()
                
                    !pjsub submit.pbs
                    time.sleep(1) # Safety in submission of jobs: can cause problems if too fast

# RESEARCH NOTES AND TESTING AREA (WILL BE DELETED UPON FINISHING)